# Regressão Linear com scikit-learn

Neste notebook realizamos regressão linear analítica para **todos os pares de variáveis** do dataset, utilizando a biblioteca scikit-learn.  
Para análise de qualidade da regressão linear, calculamos para cada par (X, y):

- **w0** (intercepto da reta)  
- **w1** (coeficiente angular)  
- **SSE** (Soma dos erros quadráticos)  
- **SST** (Soma total dos quadrados)  
- **SSR** (Soma dos quadrados da regressão)  
- **R²** (coeficiente de determinação)  
- **MSE** (erro quadrático médio)  
- **N** (número de amostras)

### Métodos de Regressão Linear

No **Exercício 01** implementamos três abordagens distintas para regressão linear, com diferenças fundamentais na forma de obtenção dos parâmetros do modelo:

1. **Regressão Linear com Scikit-Learn (`este notebook`)**  
   - Utiliza a classe `LinearRegression` da biblioteca **scikit-learn**.  
   - O ajuste é realizado por meio da **equação normal**:  
     
     $$
     \hat{\beta} = (X^TX)^{-1}X^Ty
     $$
     
   - Este é um **método analítico**, também chamado de solução fechada, que não envolve processo iterativo de otimização.  
   - Sua principal limitação é o custo computacional elevado quando o número de variáveis explicativas é muito grande, pois depende da inversão da matriz $X^TX$.

2. **Regressão Linear Interativa com BGD (`regressao_linear_interativa_BGD.ipynb`)**  
   - Implementa o **Batch Gradient Descent (BGD)**.  
   - Os parâmetros do modelo são ajustados iterativamente, a cada época, de acordo com a direção do gradiente.  
   - É um **método iterativo**, cujo resultado depende da taxa de aprendizado (learning rate), número de épocas e critérios de convergência.

3. **Regressão Linear Interativa com SGD (`regressao_linear_interativa_SGD.ipynb`)**  
   - Implementa o **Stochastic Gradient Descent (SGD)**.  
   - Em vez de considerar todo o conjunto de dados por época (como no BGD), o ajuste é feito com base em **amostras individuais** ou pequenos lotes.  
   - Também é um **método iterativo**, sujeito a oscilações, mas costuma convergir mais rapidamente em grandes conjuntos de dados.


### Carregamento do dataset

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA_PATH = Path.cwd().joinpath(Path("Exercicio01/dataset/data_0460_5832.csv"))
assert DATA_PATH.exists(), f"Arquivo não encontrado: {DATA_PATH}"

raw = pd.read_csv(DATA_PATH)
print("Dados sem tratamento:")
display(raw.head())

Dados sem tratamento:


,Sex,Age,Height,Weight,Shoe number
0,Female,53,154,59,36
1,Male,23,170,56,40
2,Female,23,167,63,37
3,Male,21,178,78,40
4,Female,25,153,58,36


### Tratamento dos dados

In [7]:
import pandas as pd
import numpy as np

# 1) nomes limpos
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (df.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(r"\s+", "_", regex=True))
    return df

# 2) tentar converter texto "numérico" para número (mantendo só colunas que realmente viram número)
def coerce_numerics(df: pd.DataFrame, min_numeric_ratio: float = 0.60) -> pd.DataFrame:
    df = df.copy()
    for c in df.select_dtypes(include="object").columns:
        s = (df[c].astype(str)
                .str.replace(r"[^\d,\-\.]", "", regex=True)   # tira símbolos
                .str.replace(",", ".", regex=False))          # vírgula -> ponto
        num = pd.to_numeric(s, errors="coerce")
        # só substitui se a maioria virou número
        if num.notna().mean() >= min_numeric_ratio:
            df[c] = num
    # se alguma coluna ficou TODA NaN depois de coerção, remove
    df = df.dropna(axis=1, how="all")
    return df

# 3) preencher ausências
def fill_missing(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].fillna(df[c].mean())
        else:
            if df[c].isna().any():
                df[c] = df[c].fillna(df[c].mode().iloc[0])
    return df

# 4) codificar categóricas (mapa dedicado para sex/gender + one-hot nas demais)
def encode_categoricals(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # padroniza e mapeia sex/gender primeiro (evita NaN por mapeamento)
    sex_maps = {"female": 0, "f": 0, "woman": 0, "male": 1, "m": 1, "man": 1}
    for col in df.columns:
        if col in {"sex", "gender"} and df[col].dtype == "object":
            std = df[col].astype(str).str.strip().str.lower()
            mapped = std.map(sex_maps)
            # se grande parte foi mapeada, usa o mapeado; senão deixa para o one-hot
            if mapped.notna().mean() >= 0.6:
                df[col] = mapped.astype("float64")  # numérico, permite fill de NaN
            else:
                # não mapeou bem? deixa como object para dummificar
                pass

    # dummies para as demais categóricas
    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    if cat_cols:
        # drop_first=False para manter TODAS as categorias (bom para pares x-y)
        df = pd.get_dummies(df, columns=cat_cols, drop_first=False, dtype="uint8")

    return df

# 5) pipeline completo — ordem ajustada para não gerar NaN por mapeamento
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df1 = clean_columns(df)
    df2 = coerce_numerics(df1)        # primeiro, tenta converter textos numéricos
    df3 = encode_categoricals(df2)    # depois transforma categóricas em números
    df4 = fill_missing(df3)           # por fim, preenche ausências
    # segurança: remove colunas ainda 100% NaN, se existirem
    df4 = df4.dropna(axis=1, how="all")
    return df4

# --- uso ---
df = preprocess(raw)

print("Depois do tratamento (apenas numérico):")
display(df.head)
df.info()

Depois do tratamento (apenas numérico):


<bound method NDFrame.head of      sex  age  height  weight  shoe_number
0    0.0   53     154      59           36
1    1.0   23     170      56           40
2    0.0   23     167      63           37
3    1.0   21     178      78           40
4    0.0   25     153      58           36
..   ...  ...     ...     ...          ...
252  1.0   71     167      83           39
253  0.0    3      12      11           21
254  1.0   25     160      55           39
255  0.0   26     164      58           38
256  1.0   27     184      72           42

[257 rows x 5 columns]>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sex          257 non-null    float64
 1   age          257 non-null    int64  
 2   height       257 non-null    int64  
 3   weight       257 non-null    int64  
 4   shoe_number  257 non-null    int64  
dtypes: float64(1), int64(4)
memory usage: 10.2 KB


### Regressão linear

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import time

KIND = "Scikit"  

def fit_pair(x: pd.Series, y: pd.Series):
    # 1-D, sem NaN/Inf
    x_arr = np.asarray(x, dtype=float).ravel()
    y_arr = np.asarray(y, dtype=float).ravel()
    mask  = np.isfinite(x_arr) & np.isfinite(y_arr)
    x_arr = x_arr[mask]; y_arr = y_arr[mask]
    if x_arr.size < 2 or y_arr.var() == 0:
        return None

    # mede tempo
    t0 = time.perf_counter()

    X = x_arr.reshape(-1, 1)
    model = LinearRegression().fit(X, y_arr)
    y_hat = model.predict(X)

    wall_s = time.perf_counter() - t0

    sse = float(np.sum((y_arr - y_hat)**2))
    sst = float(np.sum((y_arr - y_arr.mean())**2))
    ssr = float(sst - sse)
    r2  = float(1 - sse/sst) if sst > 0 else np.nan
    mse = float(mean_squared_error(y_arr, y_hat))

    return {
        "model": model,
        "w0": float(model.intercept_), "w1": float(model.coef_[0]),
        "SSE": sse, "SST": sst, "SSR": ssr,
        "R2": r2, "MSE": mse, "N": int(y_arr.size),
        "wall_s": wall_s
    }

# loop de pares
results = []
models  = {}
cols = list(df.columns)

for y_name in cols:
    for x_name in cols:
        if x_name == y_name:
            continue
        res = fit_pair(df[x_name], df[y_name])
        if res is None:
            continue
        results.append({
            "kind": KIND,                           # <-- aqui o método
            "x": x_name, "y": y_name,
            "pair": f"{x_name} → {y_name}",         # (opcional) rótulo do par
            "w0": res["w0"], "w1": res["w1"],
            "SSE": res["SSE"], "SST": res["SST"], "SSR": res["SSR"],
            "R2": res["R2"], "MSE": res["MSE"], "N": res["N"],
            "wall_s": res["wall_s"]
        })
        models[(x_name, y_name)] = res["model"]

res_df = pd.DataFrame(results).sort_values("R2", ascending=False).reset_index(drop=True)
display(res_df)

,kind,x,y,pair,w0,w1,SSE,SST,SSR,R2,MSE,N,wall_s
0,Scikit,shoe_number,height,shoe_number → height,10.121546,4.053376,15496.844981,53618.070039,38121.225058,0.710977,60.299008,257,0.000918
1,Scikit,height,shoe_number,height → shoe_number,9.639357,0.175404,670.602632,2320.241245,1649.638613,0.710977,2.609349,257,0.000889
2,Scikit,shoe_number,sex,shoe_number → sex,-3.485925,0.105604,29.594910,55.470817,25.875907,0.466478,0.115155,257,0.001110
3,Scikit,sex,shoe_number,sex → shoe_number,36.469136,4.417228,1237.900112,2320.241245,1082.341133,0.466478,4.816732,257,0.000885
4,Scikit,weight,shoe_number,weight → shoe_number,31.401738,0.114986,1437.945414,2320.241245,882.295831,0.380260,5.595118,257,0.000897
5,Scikit,shoe_number,weight,shoe_number → weight,-60.230249,3.307012,41355.465600,66730.389105,25374.923505,0.380260,160.916209,257,0.000862
6,Scikit,weight,height,weight → height,134.042165,0.513859,35997.854588,53618.070039,17620.215451,0.328625,140.069473,257,0.000928
7,Scikit,height,weight,height → weight,-38.473387,0.639523,44801.143381,66730.389105,21929.245724,0.328625,174.323515,257,0.000887
8,Scikit,height,sex,height → sex,-2.002969,0.015791,42.100173,55.470817,13.370644,0.241039,0.163814,257,0.001007
9,Scikit,sex,height,sex → height,159.753086,15.263959,40694.010592,53618.070039,12924.059447,0.241039,158.342454,257,0.000915


## Salvando os dados

In [12]:
from pathlib import Path

# Caminho da pasta onde vamos salvar
DATA_PATH = Path.cwd().joinpath("Exercicio01/tabela")
DATA_PATH.mkdir(parents=True, exist_ok=True)  # cria se não existir

# Ordem desejada das colunas
col_order = [
    "kind", "x", "y", "w0", "w1",
    "SSE", "SSR", "SST", "R2", "MSE",
    "iters", "grad_norm", "wall_s", "peak_mb", "N"
]

# Reorganiza (somente as que realmente existem no DataFrame)
cols_present = [c for c in col_order if c in res_df.columns]
res_df = res_df[cols_present]

# Salva
results_file = DATA_PATH / "results_linear_scikit.csv"
res_df.to_csv(results_file, index=False)

print(f"Arquivo salvo em: {results_file}")
print("Colunas na ordem:", res_df.columns.tolist())

Arquivo salvo em: /home/nara/MAC5921-Deep-Learning/Exercicio01/tabela/results_linear_scikit.csv
Colunas na ordem: ['kind', 'x', 'y', 'w0', 'w1', 'SSE', 'SSR', 'SST', 'R2', 'MSE', 'wall_s', 'N']
